In [2]:
import pandas as pd
import numpy as np

<H3> Lectura archivo despachos.

In [3]:
TABASTOS2019=pd.read_csv("DESPACHOS_2019.csv")

# Elimina suministros
TABASTOS2019=TABASTOS2019.drop(TABASTOS2019[TABASTOS2019["NOM_DEPTO"]=="80-MATERIAL DE EMPAQUE"].index)

# 17529 códigos de barra

display(TABASTOS2019.head(2),TABASTOS2019.shape)

,LOCAL,FECHA_DESPACHO,NUMERO_PALLET,COD_BARRAS,UNIDAD_MANEJO,CANTIDAD,UBICACION,NOM_DIVISION,NOM_BODEGA,NOM_DEPTO,NOM_CLASIFICACION,NOM_CLASIFICACION_AUX
0,106,2019-04-21,2010411911848,722008000218,72.0,1.0,1150280001,1-ABARROTES,10-ABASTOS,10-COMESTIBLES I,"1072-ATUNES,SARDINAS,MARISCOS,CARNICOS","1072-ATUNES,SARDINAS,MARISCOS,CARNICOS"
1,109,2019-04-21,2010411745887,7862106454803,27.0,2.0,4020030001,1-ABARROTES,10-ABASTOS,11-JUGOS Y BEBIDAS,1116-COCOA,1116-COCOA


(6334327, 12)

<h3> Lista de clasificaciones Contaminant

In [4]:
lista_cont = [
"3008-CLOROS",
"3012-PRODUCTOS PARA CALZADO","3013-(I) PRODUCTOS PARA CALZADO",
"3016-DESINFECTANTES",
"3018-DESODORANTES AMBIENTALES","3019-(I) DESODORANTES AMBIENTALES",
"3020-DETERGENTE","3021-(I) DETERGENTE",
"3028-LIMPIADORES VARIOS","3029-(I) LIMPIADORES VARIOS",
"3030-INSECTICIDAS","3031-(I) INSECTICIDAS",
"3032-JABON PARA LAVAR ROPA",
"3034-LAVAVAJILLAS","3035-(I) LAVAVAJILLAS",                      
"3040-CUIDADO DE LA ROPA","3041-(I) CUIDADO DE LA ROPA",
"4118-DESODORANTES","4119-(I) DESODORANTES",  
"4126-JABON DE TOCADOR","4127-(I) JABON DE TOCADOR",
"4130-LACAS Y FIJADORES",
"4138-SHAMPOOS","4139-(I) SHAMPOOS",
"4140-TALCOS Y SIMILARES","4140-(I) TALCOS Y SIMILARES",
"4162-TRATAMIENTOS CAPILARES","4163-(I) TRATAMIENTOS CAPILARES",  
"6516-ACCESORIOS LIMPIEZA MASCOTAS","6517-(I) ACCESORIOS LIMPIEZA MASCOTAS",             
"2064-ABONOS",
"2070-CARBON",
"3006-DESTAPA CANOS",
"3010-CERA DE PISOS","3011-(I) CERA DE PISOS",
"2072-DESINFECTANTE DE HORTALIZAS",
"4258-SPA",
"4122-HIGIENE BUCAL",
"4104-ACONDICIONADOR / RINSE", "4105-(I) ACONDICIONADOR / RINSE",
"4144-HIGIENE INFANTIL", "4145-(I) HIGIENE INFANTIL",
"6824-LLANTAS",
"6858-LLANTAS PARA MOTOS",    
"4228-LINEA BEBE",
"4102-EXHIBIDORES DE CAJA HIGIENE",
"4204-BRONCEADORES"]

<h1> ABCD GLOBAL

In [5]:
#ELIMINA CONTAMINANTES
GLOBAL=TABASTOS2019[~TABASTOS2019.NOM_CLASIFICACION.isin(lista_cont)]

# Frecuencia de despeachos de cada CB
cbT=pd.DataFrame({'cant':GLOBAL.groupby("COD_BARRAS").size()}).reset_index()

# Ordenando
cbT=cbT.sort_values(by="cant", ascending=False).reset_index()
del cbT["index"]

# Total de frecuencias de despachos
Freq_desT=cbT.cant.sum(axis=0)

# Participacion
cbT['PARTICIPACION']=cbT["cant"].apply(lambda par: (par/Freq_desT)*100)

# SUMA ACUMULADA
cbT['SUM_ACUM'] = cbT['PARTICIPACION'].cumsum() 

#CLASIFICACION ABCD
cbT["CLASIFICACION"]=cbT["SUM_ACUM"].apply(
    lambda x: "A" if x<=80.01 else ("B" if x>80.01 and x<=90.001 else ("C" if x>90.001 and x<=98.0 else "D")))


In [5]:
display(cbT.tail(3),cbT.shape)

,COD_BARRAS,cant,PARTICIPACION,SUM_ACUM,CLASIFICACION
13592,7798119220015,1,0.000019,99.999961,D
13593,7798131200828,1,0.000019,99.999981,D
13594,715126600033,1,0.000019,100.000000,D


(13595, 5)

In [6]:
ABCD_global=pd.DataFrame({'Total':cbT.groupby("CLASIFICACION").size()}).reset_index()
ABCD_global

,CLASIFICACION,Total
0,A,3696
1,B,2074
2,C,3776
3,D,4049


<H1> ABCD CONTAMINANTES

In [7]:
CONT=TABASTOS2019[TABASTOS2019.NOM_CLASIFICACION.isin(lista_cont)]

# Frecuencia de despachos de cada CB
cb_contaminant=pd.DataFrame({'cant':CONT.groupby("COD_BARRAS").size()}).reset_index()

# Ordenando
cb_contaminant=cb_contaminant.sort_values(by="cant", ascending=False).reset_index()
del cb_contaminant["index"]

# Total de frecuencias de despachos
Freq_des_cont=cb_contaminant.cant.sum(axis=0)

# Participacion
cb_contaminant['PARTICIPACION']=cb_contaminant["cant"].apply(lambda par: (par/Freq_des_cont)*100)

# Suma Acumulada
cb_contaminant['SUM_ACUM'] = cb_contaminant['PARTICIPACION'].cumsum() 

# CLASIFICACION ABCD
cb_contaminant["CLASIFICACION"]=cb_contaminant["SUM_ACUM"].apply(
    lambda x: "A" if x<=80.01 else ("B" if x>80.01 and x<=90.001 else ("C" if x>90.001 and x<=98 else "D")))

In [8]:
display(cb_contaminant.tail(3),cb_contaminant.shape)

,COD_BARRAS,cant,PARTICIPACION,SUM_ACUM,CLASIFICACION
3931,7509552909678,1,0.000087,99.999825,D
3932,900130890,1,0.000087,99.999913,D
3933,7804920010551,1,0.000087,100.000000,D


(3934, 5)

In [1]:
ABCD_cont=pd.DataFrame({'Total':cb_contaminant.groupby("CLASIFICACION").size()}).reset_index()
ABCD_cont

NameError: name 'pd' is not defined

In [ ]:
df = (
    pd.DataFrame.from_dict(company_sales)
    .remove_columns(['Company1'])
    .dropna(subset=['Company2', 'Company3'])
    .rename_column('Company2', 'Amazon')
    .rename_column('Company3', 'Facebook')
    .add_column('Google', [450.0, 550.0, 800.0])
)

<H1> RESISTENCIAS

In [10]:
cb_resis=pd.DataFrame({'cant':TABASTOS2019.groupby(["COD_BARRAS","NOM_DIVISION","NOM_BODEGA","NOM_DEPTO","NOM_CLASIFICACION_AUX"]).size()}).reset_index()
del cb_resis["cant"]
display(cb_resis.head(2),cb_resis.shape)

,COD_BARRAS,NOM_DIVISION,NOM_BODEGA,NOM_DEPTO,NOM_CLASIFICACION_AUX
0,60715,1-ABARROTES,10-ABASTOS,12-COMESTIBLES II,1240-VEGETALES A GRANEL
1,60763,1-ABARROTES,10-ABASTOS,12-COMESTIBLES II,1240-VEGETALES A GRANEL


(17529, 5)

In [11]:
Res_Clas=pd.read_csv("../Resistencias  SIM_6JUN /RES_CLAS.csv")
display(Res_Clas.head(2),Res_Clas.shape)


,NOM_CLASIFICACION_AUX,resis_clas
0,6824-LLANTAS,4.0
1,1314-ESPUMANTES,4.0


(91, 2)

In [12]:
cb_resis_clas=pd.merge(cb_resis,Res_Clas,on="NOM_CLASIFICACION_AUX", how="left")
display(cb_resis_clas.head(2),cb_resis_clas.shape)

,COD_BARRAS,NOM_DIVISION,NOM_BODEGA,NOM_DEPTO,NOM_CLASIFICACION_AUX,resis_clas
0,60715,1-ABARROTES,10-ABASTOS,12-COMESTIBLES II,1240-VEGETALES A GRANEL,NaN
1,60763,1-ABARROTES,10-ABASTOS,12-COMESTIBLES II,1240-VEGETALES A GRANEL,NaN


(17529, 6)

In [13]:
# Nulos
nulos_clas=cb_resis_clas[pd.isnull(cb_resis_clas.resis_clas)]
cb_resis_clas.drop(nulos_clas.index, inplace=True)

# TABLA NECESARIA
CB1=cb_resis_clas[["COD_BARRAS","resis_clas"]]

CB1.rename({"resis_clas":"RESISTENCIA"}, axis=1,inplace=True)

display(cb_resis_clas.head(3),cb_resis_clas.shape)

/home/jupyter/.local/lib/python3.5/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,COD_BARRAS,NOM_DIVISION,NOM_BODEGA,NOM_DEPTO,NOM_CLASIFICACION_AUX,resis_clas
35,69884,1-ABARROTES,10-ABASTOS,10-COMESTIBLES I,1010-ARROZ,4.0
40,40144016,1-ABARROTES,10-ABASTOS,10-COMESTIBLES I,1076-PRODUCTOS EXHIBIDORES CAJA,2.0
41,40198095,1-ABARROTES,10-ABASTOS,10-COMESTIBLES I,1006-SALSAS LISTAS/DIPS,3.0


(15207, 6)

In [14]:
RES_DPTO2=pd.read_csv("../Resistencias  SIM_6JUN /RES_DPTO2.csv")

nulos_clas_dpto=pd.merge(nulos_clas,RES_DPTO2, on="NOM_DEPTO", how="left")

# Nulos
nulos_dpto=nulos_clas_dpto[pd.isnull(nulos_clas_dpto.resis_dpto)]
nulos_clas_dpto.drop(nulos_dpto.index, inplace=True)

# TABLA NECESARIA
CB2=nulos_clas_dpto[["COD_BARRAS","resis_dpto"]]
CB2.rename({"resis_dpto":"RESISTENCIA"}, axis=1,inplace=True)

display(nulos_clas_dpto.head(3),nulos_clas_dpto.shape)

/home/jupyter/.local/lib/python3.5/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,COD_BARRAS,NOM_DIVISION,NOM_BODEGA,NOM_DEPTO,NOM_CLASIFICACION_AUX,resis_clas,resis_dpto
36,40122342,1-ABARROTES,10-ABASTOS,10-COMESTIBLES I,1066-MOSTAZAS,NaN,3.0
37,40122380,1-ABARROTES,10-ABASTOS,10-COMESTIBLES I,1066-MOSTAZAS,NaN,3.0
38,40122526,1-ABARROTES,10-ABASTOS,10-COMESTIBLES I,1066-MOSTAZAS,NaN,3.0


(2042, 7)

In [15]:
RES_DIVISION2=pd.read_csv("../Resistencias  SIM_6JUN /RES_DIVISION2.csv")

res_div=pd.merge(nulos_dpto,RES_DIVISION2,on="NOM_DIVISION",how="left")

# Nulos
nulos_div=res_div[pd.isnull(res_div.resis_division)]
res_div.drop(nulos_div.index, inplace=True)

# TABLA NECESARIA
CB3=res_div[["COD_BARRAS","resis_division"]]
CB3.rename({"resis_division":"RESISTENCIA"}, axis=1,inplace=True)

display(res_div.head(2),res_div.shape)

/home/jupyter/.local/lib/python3.5/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,COD_BARRAS,NOM_DIVISION,NOM_BODEGA,NOM_DEPTO,NOM_CLASIFICACION_AUX,resis_clas,resis_dpto,resis_division
0,60715,1-ABARROTES,10-ABASTOS,12-COMESTIBLES II,1240-VEGETALES A GRANEL,NaN,NaN,4.0
1,60763,1-ABARROTES,10-ABASTOS,12-COMESTIBLES II,1240-VEGETALES A GRANEL,NaN,NaN,4.0


(204, 8)

In [16]:
# Poniendo resistencia a los nulos (almohadas)
nulos_div["resis_division2"]=nulos_div["resis_division"].apply( lambda x: 2)

# TABLA NECESARIA
CB4=nulos_div[["COD_BARRAS","resis_division2"]]
CB4.rename({"resis_division2":"RESISTENCIA"}, axis=1,inplace=True)

nulos_div.head(2)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jupyter/.local/lib/python3.5/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,COD_BARRAS,NOM_DIVISION,NOM_BODEGA,NOM_DEPTO,NOM_CLASIFICACION_AUX,resis_clas,resis_dpto,resis_division,resis_division2
47,840101574938,7-TEXTILES,10-ABASTOS,70-LENCERIA Y MERCERIA,7016-ALMOHADAS,NaN,NaN,NaN,2
48,840101574945,7-TEXTILES,10-ABASTOS,70-LENCERIA Y MERCERIA,7016-ALMOHADAS,NaN,NaN,NaN,2


In [17]:
DESP2019_resis=pd.concat([CB1,CB2,CB3,CB4]).reset_index()
del DESP2019_resis["index"]
display(DESP2019_resis.head(),DESP2019_resis.shape)

,COD_BARRAS,RESISTENCIA
0,69884,4.0
1,40144016,2.0
2,40198095,3.0
3,40198132,3.0
4,40198569,4.0


(17529, 2)

<H1> REUBICACIÓN

<H3> Ubicaciones (1 nivel)

In [100]:
ubicacion_1N=pd.read_csv('UBICACIONES_1NIVEL.csv',sep=";",usecols=[0,2,3])
ubicacion_1N.drop(ubicacion_1N[ubicacion_1N.ZONA=="SMN"].index, inplace=True)

# Ubicaciones necesarias en 2 nivel
racks_2n=pd.read_excel('RACKS_2DO nivel.xlsx',sep=";",usecols=[0,2,3])

# Unir ubicaicones 1 nivel y 2 nivel
ubicaciones_1N=pd.concat([ubicacion_1N,racks_2n])


# CONTAMINANTES (Nave 5)
ubi_cont1N=ubicaciones_1N[ubicaciones_1N["NAVE"]==5]

# CONTAMINABLES (Nave 1,2,3 y 4)
ubi_NOcont1N=ubicaciones_1N.loc[ubicaciones_1N['NAVE'].isin([1,2,3,4])]

display(ubi_cont1N.shape,ubi_NOcont1N.shape)

(3951, 3)

(13620, 3)

<H3> Ubicaciones (2 niveles)

In [45]:
ubicaciones=pd.read_csv('UBICACION-ZONA-2N.csv',sep=";",usecols=[0,2,3])
ubicaciones.drop(ubicaciones[ubicaciones.ZONA=="SMN"].index, inplace=True)

# CONTAMINANTES (Nave 5)
ubi_cont=ubicaciones[ubicaciones["NAVE"]==5]

# CONTAMINABLES (Nave 1,2,3 y 4)
ubi_NOcont=ubicaciones.loc[ubicaciones['NAVE'].isin([1,2,3,4])]

display(ubicaciones.head(2),ubicaciones.shape)

,NAVE,UBICACION,ZONA
0,1,1140480001,C
1,1,1140470001,C


(31128, 3)

In [ ]:
# CONTAMINANTES (Nave 5)
ubi_cont=ubicaciones[ubicaciones["NAVE"]==5]

# CONTAMINABLES (Nave 1,2,3 y 4)
ubi_NOcont=ubicaciones.loc[ubicaciones['NAVE'].isin([1,2,3,4])]

display(ubicaciones.head(2),ubicaciones.shape)

<h2> CONTAMINANTES
    
<h4> Datos necesarios de contaminantes
        

In [46]:
contaminante=pd.merge(cb_contaminant,DESP2019_resis,on="COD_BARRAS")

Contaminante=contaminante[["COD_BARRAS","CLASIFICACION","RESISTENCIA"]]

display(Contaminante.head(2),Contaminante.shape)

,COD_BARRAS,CLASIFICACION,RESISTENCIA
0,7702010780363,A,3.0
1,7861001350197,A,3.0


(3934, 3)

<h4> Organizando resistencias y ubicaciones para reubicar

In [74]:
# CODIGOS DE BARRA
cont_resA=Contaminante[Contaminante["CLASIFICACION"]=="A"].sort_values(by="RESISTENCIA", ascending=False).reset_index()
del cont_resA["index"]

cont_resB=Contaminante[Contaminante["CLASIFICACION"]=="B"].sort_values(by="RESISTENCIA", ascending=False).reset_index()
del cont_resB["index"]

cont_resC=Contaminante[Contaminante["CLASIFICACION"]=="C"].sort_values(by="RESISTENCIA", ascending=False).reset_index()
del cont_resC["index"]

cont_resD=Contaminante[Contaminante["CLASIFICACION"]=="D"].sort_values(by="RESISTENCIA", ascending=False).reset_index()
del cont_resD["index"]


#UBICACIONES 
cont_ubiA=ubi_cont1N[ubi_cont1N["ZONA"]=="A"].sort_values(by="UBICACION", ascending=True).reset_index()
del cont_ubiA["index"]

cont_ubiB=ubi_cont1N[ubi_cont1N["ZONA"]=="B"].sort_values(by="UBICACION", ascending=True).reset_index()
del cont_ubiB["index"]

cont_ubiC=ubi_cont1N[ubi_cont1N["ZONA"]=="C"].sort_values(by="UBICACION", ascending=True).reset_index()
del cont_ubiC["index"]

cont_ubiD=ubi_cont1N[ubi_cont1N["ZONA"]=="D"].sort_values(by="UBICACION", ascending=True).reset_index()
del cont_ubiD["index"]

<h3> Zona A: Zona - Reubicación

In [75]:
cont_ubiA['indice']=cont_ubiA.index 
cont_resA['indice']=cont_resA.index

# Asignación de cb a ubicaciones
cb_ubi_A=pd.merge(cont_resA,cont_ubiA, on="indice", how="inner")

ub_cont_A=cb_ubi_A[["COD_BARRAS","RESISTENCIA","UBICACION","ZONA"]]

display(ub_cont_A.head(2),ub_cont_A.shape)

,COD_BARRAS,RESISTENCIA,UBICACION,ZONA
0,51342179077,4.0,5010210001,A
1,51342201204,4.0,5010220001,A


(1348, 4)

<h3> Zona B: Zona - Reubicación

In [76]:
# ZONA UBICACIONES "B"
cont_ubiB['indice']=cont_ubiB.index 
cont_resB['indice']=cont_resB.index

cb_ubi_B=pd.merge(cont_resB,cont_ubiB, on="indice", how="inner")

ub_cont_B=cb_ubi_B[["COD_BARRAS","RESISTENCIA","UBICACION","ZONA"]]

display(ub_cont_B.head(2),ub_cont_B.shape)

,COD_BARRAS,RESISTENCIA,UBICACION,ZONA
0,51342201228,4.0,5010090001,B
1,51342201242,4.0,5010100001,B


(648, 4)

<h3> Zona C: Zona - Reubicación

In [77]:
# ZONA UBICACIONES "C"
cont_ubiC['indice']=cont_ubiC.index 
cont_resC['indice']=cont_resC.index

cb_ubi_C=pd.merge(cont_resC,cont_ubiC, on="indice", how="inner")

ub_cont_C=cb_ubi_C[["COD_BARRAS","RESISTENCIA","UBICACION","ZONA"]]
display(ub_cont_C.head(2),ub_cont_C.shape)

,COD_BARRAS,RESISTENCIA,UBICACION,ZONA
0,51342179145,4.0,5010010001,C
1,51342179121,4.0,5010011001,C


(1052, 4)

<h3> Zona D: Zona - Reubicación

In [78]:
cont_ubiD['indice']=cont_ubiD.index 
cont_resD['indice']=cont_resD.index

cb_ubi_D=pd.merge(cont_resD,cont_ubiD, on="indice", how="inner")

ub_cont_D=cb_ubi_D[["COD_BARRAS","RESISTENCIA","UBICACION","ZONA"]]
display(ub_cont_D.head(2),ub_cont_D.shape)

,COD_BARRAS,RESISTENCIA,UBICACION,ZONA
0,51342191383,4.0,5270230001,D
1,51342206964,4.0,5270230010,D


(886, 4)

<h3> Uniendo reubicación de zonas en uno solo de Contaminantes

In [79]:
REUBI_CONT=pd.concat([ub_cont_A,ub_cont_B,ub_cont_C,ub_cont_D])
display(REUBI_CONT.head(2),REUBI_CONT.shape)

,COD_BARRAS,RESISTENCIA,UBICACION,ZONA
0,51342179077,4.0,5010210001,A
1,51342201204,4.0,5010220001,A


(3934, 4)

<h2> NO CONTAMINANTES
    
<h4> Datos necesarios de contaminantes
        

In [80]:
NO_contaminante=pd.merge(cbT,DESP2019_resis,on="COD_BARRAS")

NO_Contaminante=NO_contaminante[["COD_BARRAS","CLASIFICACION","RESISTENCIA"]]

display(NO_Contaminante.head(2),NO_Contaminante.shape)

,COD_BARRAS,CLASIFICACION,RESISTENCIA
0,7861024607674,A,4.0
1,759494000323,A,4.0


(13595, 3)

<h4> Organizando resistencias y ubicaciones para reubicar

In [101]:
# CODIGOS DE BARRA
NOcont_resA=NO_Contaminante[NO_Contaminante["CLASIFICACION"]=="A"].sort_values(by="RESISTENCIA", ascending=False).reset_index()
del NOcont_resA["index"]

NOcont_resB=NO_Contaminante[NO_Contaminante["CLASIFICACION"]=="B"].sort_values(by="RESISTENCIA", ascending=False).reset_index()
del NOcont_resB["index"]

NOcont_resC=NO_Contaminante[NO_Contaminante["CLASIFICACION"]=="C"].sort_values(by="RESISTENCIA", ascending=False).reset_index()
del NOcont_resC["index"]

NOcont_resD=NO_Contaminante[NO_Contaminante["CLASIFICACION"]=="D"].sort_values(by="RESISTENCIA", ascending=False).reset_index()
del NOcont_resD["index"]


#UBICACIONES
# ZONA A
A=ubi_NOcont1N[ubi_NOcont1N["ZONA"]=="A"]
A1=A[A["NAVE"]==1].sort_values(by="UBICACION", ascending=True)
A234=A.loc[A['NAVE'].isin([2,3,4])].sort_values(by="UBICACION", ascending=False)
NOcont_ubiA=pd.concat([A1,A234]).reset_index()
del NOcont_ubiA["index"]

# ZONA B
B=ubi_NOcont1N[ubi_NOcont1N["ZONA"]=="B"]
B1=B[B["NAVE"]==1].sort_values(by="UBICACION", ascending=True)
B234=B.loc[B['NAVE'].isin([2,3,4])].sort_values(by="UBICACION", ascending=False)
NOcont_ubiB=pd.concat([B1,B234]).reset_index()
del NOcont_ubiB["index"]

# ZONA C
C=ubi_NOcont1N[ubi_NOcont1N["ZONA"]=="C"]
C1=C[C["NAVE"]==1].sort_values(by="UBICACION", ascending=True)
C234=C.loc[C['NAVE'].isin([2,3,4])].sort_values(by="UBICACION", ascending=False)
NOcont_ubiC=pd.concat([C1,C234]).reset_index()
del NOcont_ubiC["index"]

# ZONA D
D=ubi_NOcont1N[ubi_NOcont1N["ZONA"]=="D"]
D1=D[D["NAVE"]==1].sort_values(by="UBICACION", ascending=True)
D234=D.loc[D['NAVE'].isin([2,3,4])].sort_values(by="UBICACION", ascending=False)
NOcont_ubiD=pd.concat([D1,D234]).reset_index()
del NOcont_ubiD["index"]

<h3> Zona A: Zona - Reubicación

In [102]:
NOcont_ubiA['indice']=NOcont_ubiA.index 
NOcont_resA['indice']=NOcont_resA.index

# Asignación de cb a ubicaciones
NCcb_ubi_A=pd.merge(NOcont_resA,NOcont_ubiA, on="indice", how="inner")

NCub_cont_A=NCcb_ubi_A[["COD_BARRAS","RESISTENCIA","UBICACION","ZONA"]]

display(NCub_cont_A.head(2),NCub_cont_A.shape)

,COD_BARRAS,RESISTENCIA,UBICACION,ZONA
0,7861024607674,4.0,1140010001,A
1,7861042516804,4.0,1140011001,A


(3696, 4)

<h3> Zona  B: Zona - Reubicación

In [103]:
NOcont_ubiB['indice']=NOcont_ubiB.index 
NOcont_resB['indice']=NOcont_resB.index

# Asignación de cb a ubicaciones
NCcb_ubi_B=pd.merge(NOcont_resB,NOcont_ubiB, on="indice", how="inner")

NCub_cont_B=NCcb_ubi_B[["COD_BARRAS","RESISTENCIA","UBICACION","ZONA"]]

display(NCub_cont_B.head(2),NCub_cont_B.shape)

,COD_BARRAS,RESISTENCIA,UBICACION,ZONA
0,7861030531024,4.0,1140110001,B
1,794166004605,4.0,1140110010,B


(2074, 4)

<h3> Zona C: Zona - Reubicación

In [104]:
NOcont_ubiC['indice']=NOcont_ubiC.index 
NOcont_resC['indice']=NOcont_resC.index

# Asignación de cb a ubicaciones
NCcb_ubi_C=pd.merge(NOcont_resC,NOcont_ubiC, on="indice", how="inner")

NCub_cont_C=NCcb_ubi_C[["COD_BARRAS","RESISTENCIA","UBICACION","ZONA"]]

display(NCub_cont_C.head(2),NCub_cont_C.shape)

,COD_BARRAS,RESISTENCIA,UBICACION,ZONA
0,5701116211216,4.0,1010501001,C
1,7861000261166,4.0,1010511001,C


(3776, 4)

<h3> Zona D: Zona - Reubicación

In [105]:
NOcont_ubiD['indice']=NOcont_ubiD.index 
NOcont_resD['indice']=NOcont_resD.index

# Asignación de cb a ubicaciones
NCcb_ubi_D=pd.merge(NOcont_resD,NOcont_ubiD, on="indice", how="inner")

NCub_cont_D=NCcb_ubi_D[["COD_BARRAS","RESISTENCIA","UBICACION","ZONA"]]

display(NCub_cont_D.head(2),NCub_cont_D.shape)

,COD_BARRAS,RESISTENCIA,UBICACION,ZONA
0,715126600033,4.0,1020010001,D
1,7862117760269,4.0,1020020001,D


(4049, 4)

<h3> Uniendo reubicación de zonas en uno solo de NO Contaminantes

In [107]:
REUBI_NO_CONT=pd.concat([NCub_cont_A,NCub_cont_B,NCub_cont_C,NCub_cont_D])
display(REUBI_NO_CONT.head(2),REUBI_NO_CONT.shape)

,COD_BARRAS,RESISTENCIA,UBICACION,ZONA
0,7861024607674,4.0,1140010001,A
1,7861042516804,4.0,1140011001,A


(13595, 4)

<H3> FINAL REUBICACION

In [108]:
REUBICACION_MAM2019=pd.concat([REUBI_CONT,REUBI_NO_CONT])
REUBICACION_MAM2019.rename({"UBICACION":"REUBICACION"},axis=1,inplace=True)
display(REUBICACION_MAM2019.head(2),REUBICACION_MAM2019.shape)

,COD_BARRAS,RESISTENCIA,REUBICACION,ZONA
0,51342179077,4.0,5010210001,A
1,51342201204,4.0,5010220001,A


(17529, 4)

In [109]:
REUBICACION_MAM2019.to_csv("REUBICACION_MAM2019_MODI.csv",index=False,header=True) # CON ciertos racks de nivel 2

In [3]:
REUBICACION_MAM2019=pd.read_csv("REUBICACION_MAM2019_MODI.csv")

In [8]:
REUBICACION_MAM2019.head()

,COD_BARRAS,RESISTENCIA,REUBICACION,ZONA,NAVE
0,51342179077,4.0,5010210001,A,5
1,51342201204,4.0,5010220001,A,5
2,51342201198,4.0,5010230001,A,5
3,51342201143,4.0,5010240001,A,5
4,51342201167,4.0,5010250001,A,5


In [7]:
REUBICACION_MAM2019['NAVE']=REUBICACION_MAM2019['REUBICACION'].map(lambda x: str(x)[:1]) 

In [14]:
REUBICACION_MAM2019.groupby("NAVE").size()

NAVE
1    2828
2    4188
3    4203
4    2376
5    3934
dtype: int64